In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
def bonbubun_allocator(x):
    # x is a string
    try:
        if pd.isna(x) == True:
            return x
        else:
            return str(int(x))
    except:
        return np.nan

In [3]:
def buildinggroup_modifier(x):
    # x is a string
    
    try:
        return int(x)
    except:
        return x

In [4]:
def add_0(x):
    if pd.isnull(x) == False:
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x
    else:
        return x

In [5]:
def read_coor_data4():
    basedir = './좌표데이터4/'
    filenames = [f for f in os.listdir(basedir) if (f.endswith('.txt'))&(f.startswith('entrc_seoul'))]
    
    colnames = ['시군구코드', '출입구일련번호', '법정동코드', '시도명', '시군구명', '읍면동명', '도로명코드', '도로명',
                '지하여부', '건물본번', '건물부번', '건물명', '우편번호', '건물용도분류', '건물군여부', '관할행정동',
                'X좌표', 'Y좌표'
               ]
    
    dfs_list = []
    for file in tqdm(filenames):
        try:
            df = pd.read_csv(basedir + file, sep='|', header=None, names=colnames, encoding='euc-kr')
        except:
            try:
                df = pd.read_csv(basedir + file, sep='|', header=None, names=colnames, encoding='ansi')
            except:
                try:
                    df = pd.read_csv(basedir + file, sep='|', header=None, names=colnames)
                except:
                    raise ValueError('myerror')
                    
                    
        dfs_list.append(df)
        
    concat_df = pd.concat(dfs_list).reset_index(drop=True).drop_duplicates()
    
    return concat_df

In [7]:
%%time
prac_coor_df = read_coor_data4()
prac_coor_df['도로명주소'] = prac_coor_df['시도명'] + ' ' + prac_coor_df['시군구명'] + ' ' + prac_coor_df['도로명'] + ' '\
+ prac_coor_df['건물본번'].apply(str) + '-' + prac_coor_df['건물부번'].apply(str)
print(prac_coor_df.shape)
prac_coor_df.head()

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:04<00:00,  1.36s/it]


(548456, 19)
Wall time: 6.21 s


,시군구코드,출입구일련번호,법정동코드,시도명,시군구명,읍면동명,도로명코드,도로명,지하여부,건물본번,건물부번,건물명,우편번호,건물용도분류,건물군여부,관할행정동,X좌표,Y좌표,도로명주소
0,11110,760,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,94,0,NaN,3047,근린생활시설,0,청운효자동,953241.683263,1.954023e+06,서울특별시 종로구 자하문로 94-0
1,11110,4933,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,96,0,평안빌,3047,주택,0,청운효자동,953243.258151,1.954034e+06,서울특별시 종로구 자하문로 96-0
2,11110,287,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,98,0,청운빌라,3047,주택,0,청운효자동,953240.877646,1.954055e+06,서울특별시 종로구 자하문로 98-0
3,11110,1336,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,99,3,풍림팍사이드빌라,3032,주택,1,청운효자동,953195.459364,1.954079e+06,서울특별시 종로구 자하문로 99-3
4,11110,6021,1111010100,서울특별시,종로구,청운동,111103100012,자하문로,0,99,4,경복빌라,3032,주택,1,청운효자동,953193.346779,1.954097e+06,서울특별시 종로구 자하문로 99-4


In [ ]:
main_data_df = pd.read_csv('./prepped_data/main_data_baseline_checkpoint_1.csv')
print(main_data_df.shape)

In [ ]:
main_data_df['도로명주소'].nunique()

In [ ]:
def roadname_addr_modifier(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        splitted = x.split('-')
        if len(splitted) == 1:
            return x + '-0'
        else:
            return x

In [ ]:
main_data_df['도로명주소'] = main_data_df['도로명주소'].apply(roadname_addr_modifier)
main_data_df['도로명주소'].unique()[:50]

In [ ]:
coor_df = prac_coor_df[['지번주소', 'X좌표', 'Y좌표']].drop_duplicates()
coor_df.shape

In [ ]:
coor_df.rename(columns={'지번주소':'도로명주소'}, inplace=True)

In [ ]:
coor_df.head()

In [ ]:
merge_df = main_data_df.merge(coor_df, on=['도로명주소'])
merge_df.shape

In [ ]:
merge_df.head()

In [ ]:
merge_df.drop(columns=['addr_split_len', '대장구분명'], inplace=True)
merge_df.head()

In [ ]:
merge_df.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
merge_df.head()

In [ ]:
merge_df.to_csv('./prepped_data/main_data_baseline_checkpoint_2.csv', index=False)